<hr>

# AI Workflow Capstone - AAVAIL - **Part 3 - Model Production**

Student: `Alexey Tyurin`<br>
Date: `1/18/2024`
<hr>

### Outline

1. Build a draft version of an API with train, predict, and logfile endpoints.
2. Using Docker, bundle your API, model, and unit tests.
3. Using test-driven development iterate on your API in a way that anticipates scale, load, and drift.
4. Create a post-production analysis script that investigates the relationship between model performance and the business metric.
5. Articulate your summarized findings in a final report.

At a higher level you are being asked to:

1. Ready your model for deployment
2. Query your API with new data and test your monitoring tools
3. Compare your results to the gold standard


### Create a flask API

The API performs the train, predict and logfile tasks

In [1]:
#here we will be using the data from the previous notebook to create different models and compare them to see which one is the best

# Importing the libraries
from utils_model import * #model_train, model_load, model_predict
import requests
import warnings

warnings.filterwarnings('ignore')

MODEL_DIR = os.path.join(".", "models")
LOG_DIR = os.path.join(".", "logs")


In [2]:
model_train()

... loading time series data from files
Training Models
Country: all ...best model: K Neighbors Regressor, rmse = 13,768.44, R^2 = 97.3%
Country: eire ...best model: Random Forest Regressor, rmse = 2,135.77, R^2 = 63.3%
Country: france ...best model: Random Forest Regressor, rmse = 347.85, R^2 = 87.5%
Country: germany ...best model: K Neighbors Regressor, rmse = 413.21, R^2 = 84.2%
Country: hong_kong ...best model: Random Forest Regressor, rmse = 683.21, R^2 = 84.1%
Country: netherlands ...best model: K Neighbors Regressor, rmse = 108.98, R^2 = 90.0%
Country: norway ...best model: Ada Boosting Regressor, rmse = 9,049.62, R^2 = -1256.9%
Country: portugal ...best model: Random Forest Regressor, rmse = 166.85, R^2 = 98.3%
Country: singapore ...best model: Ada Boosting Regressor, rmse = 2,094.88, R^2 = -114.7%
Country: spain ...best model: K Neighbors Regressor, rmse = 226.97, R^2 = 94.1%
Country: united_kingdom ...best model: K Neighbors Regressor, rmse = 17,045.27, R^2 = 95.5%


In [3]:
# load the model
all_models = model_load(model_dir='models')
print(all_models.keys())

Loading Models
dict_keys(['all', 'eire', 'france', 'germany', 'hong_kong', 'netherlands', 'norway', 'portugal', 'singapore', 'spain', 'united_kingdom'])


In [4]:
query = {'year': 2018,
         'month': 1,
         'day': 5,
         'country': 'all',
         'dev': True,
         'verbose': True}

model_predict(**query)


Make Prediction for 2018-01-05
... loading time series data from files
Loading Models


{'y_pred': array([183479.624])}

In [5]:
## API predict
import requests
from ast import literal_eval

query = {'year': 2018,
         'month': 1,
         'day': 5,
         'country': 'all',
         'dev': 'True',
         'verbose': 'True'}

port = 8080
r = requests.post(f'http://localhost:{port}/predict', json=query)
response = literal_eval(r.text)
print(r.text)

{"y_pred":[183479.62399999998]}



In [6]:
## API train
query = {'dev': 'True',
         'verbose': 'True'}
port = 8080
r = requests.post('http://localhost:{}/train'.format(port),json=query)

In [7]:
## API logging
query = {'env': 'test',
         'tag': 'train',
         'year': '2024',
         'month': '1',
         'verbose': 'True'}

port = 8080
r = requests.post('http://localhost:{}/logging'.format(port),json=query)
response = literal_eval(r.text)
print(response)


{'logfile': 'test-train-2024-1.log'}


## Create Unit Tests

The unit tests are usually organized as a suite and return objective evidence, in the form of a boolean value, which is a key element that enables workflow automation. The boolean value indicates whether or not each and every part of the software that was tested performed as expected. Much like data ingestion, the idea is to have the necessary components of a task bundled under a single script. In this case it will be called `run-tests.py`.